In [ ]:
import numpy as np
import pywt
from PIL import Image
import matplotlib.pyplot as plt
import io
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import base64

# Function to perform 2D Discrete Haar Wavelet Transform
def dhwt_2d(image):
    coeffs = pywt.dwt2(image, 'haar')
    LL, (LH, HL, HH) = coeffs
    return LL, LH, HL, HH

# Function to generate chaotic sequences
def chaotic_map(seed, iterations):
    chaotic_seq = []
    x = seed
    for _ in range(iterations):
        x = 3.9 * x * (1 - x)  # Logistic map equation
        chaotic_seq.append(x)
    return chaotic_seq

# Encryption process
def encrypt_image(image):
    # Flatten the image into a 1D array for each channel
    R, G, B = image[:,:,0], image[:,:,1], image[:,:,2]
    flat_R = R.flatten()
    flat_G = G.flatten()
    flat_B = B.flatten()
    
    # Concatenate the flattened channels
    flat_image = np.concatenate((flat_R, flat_G, flat_B))
    
    # Generate permutation keys
    iterations = len(flat_image) + 500  # Number of iterations
    seed = 0.1  # Initial seed for chaotic map
    chaotic_seq = chaotic_map(seed, iterations)
    
    # Truncate the chaotic sequence to match the length of the flattened image
    chaotic_seq = chaotic_seq[:len(flat_image)]
    
    # Apply chaos-based permutation on the flattened image
    permutation_indices = np.argsort(chaotic_seq)
    encrypted_flat_image = flat_image[permutation_indices]
    
    # Split the encrypted flattened image back into channels
    encrypted_R = encrypted_flat_image[:len(flat_R)].reshape(R.shape)
    encrypted_G = encrypted_flat_image[len(flat_R):len(flat_R) + len(flat_G)].reshape(G.shape)
    encrypted_B = encrypted_flat_image[len(flat_R) + len(flat_G):].reshape(B.shape)
    
    # Return encrypted image components, permutation keys, and original image dimensions
    return (encrypted_R, encrypted_G, encrypted_B), permutation_indices, image.shape

# Decryption process
def decrypt_image(encrypted_image, permutation_indices, original_shape):
    # Retrieve original image dimensions
    rows, cols, channels = original_shape
    
    # Flatten the encrypted image into a 1D array for each channel
    encrypted_R, encrypted_G, encrypted_B = encrypted_image
    flat_encrypted_R = encrypted_R.flatten()
    flat_encrypted_G = encrypted_G.flatten()
    flat_encrypted_B = encrypted_B.flatten()
    
    # Concatenate the flattened encrypted channels
    flat_encrypted_image = np.concatenate((flat_encrypted_R, flat_encrypted_G, flat_encrypted_B))
    
    # Reverse the permutation using the permutation indices
    decrypted_flat_image = flat_encrypted_image[np.argsort(permutation_indices)]
    
    # Split the decrypted flattened image back into channels
    decrypted_R = decrypted_flat_image[:rows*cols].reshape((rows, cols))
    decrypted_G = decrypted_flat_image[rows*cols:2*rows*cols].reshape((rows, cols))
    decrypted_B = decrypted_flat_image[2*rows*cols:].reshape((rows, cols))
    
    # Merge decrypted components into the original image shape
    decrypted_image = np.stack((decrypted_R, decrypted_G, decrypted_B), axis=-1)
    
    # Return decrypted image
    return decrypted_image

# Main code
# Load the original image
original_image = np.array(Image.open(r"/content/CJT47.jpg"))

# Encrypt the image
encrypted_image, keys, original_shape = encrypt_image(original_image)

# Load the additional image
additional_image = np.array(Image.open(r"/content/hOIuY.jpg"))

# Resize the additional image to match the shape of the encrypted image
additional_image_resized = Image.fromarray(additional_image).resize((original_image.shape[1], original_image.shape[0]))

# Convert the resized additional image to numpy array
additional_image_resized_np = np.array(additional_image_resized)

# Transpose the additional image to match the shape of the encrypted image
additional_image_transposed = np.transpose(additional_image_resized_np, (2, 0, 1))

# Overlay the additional image on the encrypted image
overlayed_image = np.maximum(encrypted_image, additional_image_transposed)

# Convert the overlayed image to PIL format
overlayed_image_pil = Image.fromarray(np.transpose(overlayed_image, (1, 2, 0)).astype(np.uint8))

# Save the overlayed image to a BytesIO object
overlayed_image_bytes = io.BytesIO()
overlayed_image_pil.save(overlayed_image_bytes, format='JPEG')
overlayed_image_bytes.seek(0)

# Email details
sender_email = "movinreddy2016@gmail.com" # Add your email here
receiver_email = "sathichandrasekharsomireddy@gmail.com@gmail.com"
subject = "Encrypted Image with Additional Layer"
body = "Here is the encrypted image with an additional layer."

# Create message container
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject

# Attach the encrypted image with additional layer
attachment = MIMEBase('application', 'octet-stream')
attachment.set_payload(overlayed_image_bytes.getvalue())
encoders.encode_base64(attachment)
attachment.add_header('Content-Disposition', f'attachment; filename=encrypted_image_with_additional_layer.jpg')
msg.attach(attachment)

# Send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    # Be sure to use an app password if using Gmail
    server.login(sender_email, "nook ffpm ieyp yiui") # Add your app password here
    server.sendmail(sender_email, receiver_email, msg.as_string())